<a href="https://colab.research.google.com/github/Remache-Abderraheem/ChatbotCollab/blob/main/rag_video_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import auth
auth.authenticate_user()


In [2]:
pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using c

In [1]:
from google.cloud import firestore
import whisper
from google.cloud import storage
from whisper.utils import get_writer
import re

whisper_bucket = "rag-videos-engine-whisper"

def is_valid_text(text):
  # Remove all special characters
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  # Split the cleaned text into words
  words = cleaned_text.split()
  # Check if the number of words exceeds 20
  if len(words) > 30:
      return True
  else:
      return False


def download_video(blob_name) -> str:

    # Initialize GCS client
    client = storage.Client()

    # Replace 'your-bucket-name' with the name of your GCS bucket
    bucket = client.bucket('rag-videos-engine-downloads')

    blob = bucket.blob(blob_name)

    blob.download_to_filename("vid.mp4")



def save_file(results, format='tsv'):
    writer = get_writer(format, './')
    writer(results, f'transcribe.{format}')


def upload_file_to_bucket(bucket_name, folder_name, file_name, file_content):
    """Uploads a file to a Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(folder_name + "/" + file_name)
    blob.upload_from_string(file_content)


# Initialize Firestore client
db = firestore.Client(project="rag-videos-engine")

# Replace 'your-collection-name' with the name of your collection
collection_name = 'video-source'

docs = db.collection(collection_name).stream()
wstatus = []
testModel = whisper.load_model("tiny")
processingModel = whisper.load_model("large")


for doc in docs:
  if doc.id == "0s1UkYLvh6vfmS6VMLtx":
    print(doc.id)
    break
doc_dict = doc.to_dict()
title_array = doc_dict.get("Title")
text = doc_dict.get("Text")
status_array = doc_dict.get("Status")
whisper_array = doc_dict.get("WStatus")
if whisper_array:
  print("no more to add")
else:
  index = 0
  for title,status in zip(title_array, status_array):
    if status == "OK":
      print(title)
      download_video(title+".mp4")
      test = testModel.transcribe("vid.mp4", task='translate')
      print(test)
      if is_valid_text(test["text"]):
          print("Valid text")
          result = processingModel.transcribe("vid.mp4", task='translate')
          types = ["txt", "vtt", "srt", "tsv", "json"]
          for ftype in types:
            save_file(result, ftype)
          folder = doc.id+"/"+str(index)
          for ftype in types:
            with open("transcribe."+ftype, 'r') as file:
              # Read the content of the file
              file_content = file.read()
            upload_file_to_bucket(whisper_bucket,folder,"transcribe."+ftype,file_content)
          wstatus.append("Transcribed")
          index+=1
      else:
          print("Invalid text")
          if is_valid_text(text):
            folder = doc.id+"/"+str(index)
            upload_file_to_bucket(whisper_bucket,folder,"transcribe.txt",text)
            wstatus.append("Empty")
            index+=1
          else:
            wstatus.append("Unuseful")
            index+=1
    else:
      wstatus.append("Error")
      index+=1
doc_ref = db.collection(collection_name).document(doc.id)
doc_ref.update({"WStatus": wstatus})

0s1UkYLvh6vfmS6VMLtx
More_and_more_crimes1
{'text': " You should know how many times it he's doing It's the song Let's go then Let me tell you See you guys In the sun Sweat! They toured at a caigy", 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.2600000000000002, 'text': " You should know how many times it he's doing", 'tokens': [50364, 509, 820, 458, 577, 867, 1413, 309, 415, 311, 884, 50527], 'temperature': 1.0, 'avg_logprob': -4.0058424813406805, 'compression_ratio': 1.2566371681415929, 'no_speech_prob': 0.052502818405628204}, {'id': 1, 'seek': 0, 'start': 5.4, 'end': 6.96, 'text': " It's the song", 'tokens': [50634, 467, 311, 264, 2153, 50712], 'temperature': 1.0, 'avg_logprob': -4.0058424813406805, 'compression_ratio': 1.2566371681415929, 'no_speech_prob': 0.052502818405628204}, {'id': 2, 'seek': 0, 'start': 7.58, 'end': 8.86, 'text': " Let's go then", 'tokens': [50743, 961, 311, 352, 550, 50807], 'temperature': 1.0, 'avg_logprob': -4.0058424813406805, 'compression_ratio

update_time {
  seconds: 1714829048
  nanos: 346918000
}